## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
file_path = os.path.join("..","Weather_Database","WeatherPy_database.csv")
city_data_df = pd.read_csv(file_path)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.66,71,14,18.97,few clouds
1,1,Jamestown,US,42.0970,-79.2353,47.79,71,100,5.75,overcast clouds
2,2,Cherskiy,RU,68.7500,161.3000,-14.60,94,8,3.56,clear sky
3,3,Mataura,NZ,-46.1927,168.8643,51.48,90,69,4.88,broken clouds
4,4,Hilo,US,19.7297,-155.0900,72.28,93,100,8.05,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,78.66,71,14,18.97,few clouds
6,6,Baculin,PH,7.4525,126.5842,79.16,83,98,5.06,light rain
12,12,Acapulco,MX,16.8634,-99.8901,75.02,83,75,0.00,broken clouds
15,15,Hambantota,LK,6.1241,81.1185,77.34,100,40,3.44,scattered clouds
16,16,Vaini,TO,-21.2000,-175.2000,77.16,94,20,8.05,few clouds
17,17,Cape Town,ZA,-33.9258,18.4232,81.12,51,0,10.36,clear sky
29,29,Touros,BR,-5.1989,-35.4608,85.46,59,100,11.14,overcast clouds
40,40,Arraial Do Cabo,BR,-22.9661,-42.0278,82.36,74,20,16.11,few clouds
43,43,Bulalacao,PH,12.3254,121.3435,76.53,80,100,6.22,overcast clouds
53,53,Cayenne,GF,4.9333,-52.3333,82.09,1,0,4.61,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                217
City                   217
Country                216
Lat                    217
Lng                    217
Max Temp               217
Humidity               217
Cloudiness             217
Wind Speed             217
Current Description    217
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.66,few clouds,-23.1203,-134.9692,
6,Baculin,PH,79.16,light rain,7.4525,126.5842,
12,Acapulco,MX,75.02,broken clouds,16.8634,-99.8901,
15,Hambantota,LK,77.34,scattered clouds,6.1241,81.1185,
16,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,
17,Cape Town,ZA,81.12,clear sky,-33.9258,18.4232,
29,Touros,BR,85.46,overcast clouds,-5.1989,-35.4608,
40,Arraial Do Cabo,BR,82.36,few clouds,-22.9661,-42.0278,
43,Bulalacao,PH,76.53,overcast clouds,12.3254,121.3435,
53,Cayenne,GF,82.09,clear sky,4.9333,-52.3333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,78.66,few clouds,-23.1203,-134.9692,People ThankYou
6,Baculin,PH,79.16,light rain,7.4525,126.5842,Baculin Cove Beach Resort
12,Acapulco,MX,75.02,broken clouds,16.8634,-99.8901,HS Hotsson Smart Acapulco
15,Hambantota,LK,77.34,scattered clouds,6.1241,81.1185,Bungalow 63
16,Vaini,TO,77.16,few clouds,-21.2000,-175.2000,Keleti Beach Resort
17,Cape Town,ZA,81.12,clear sky,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
29,Touros,BR,85.46,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON
40,Arraial Do Cabo,BR,82.36,few clouds,-22.9661,-42.0278,Pousada Porto Praia
43,Bulalacao,PH,76.53,overcast clouds,12.3254,121.3435,By The Sea Hotels
53,Cayenne,GF,82.09,clear sky,4.9333,-52.3333,Hôtel Le Dronmi


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))